# **I. Show original data folder and path**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **II. Set up data directory**

## **1. Unzip train folder to directory .**

In [ ]:
!unzip ../input/dogs-vs-cats/train.zip -d .

## **2. Split train folder to dog and cat folders**

In [ ]:
%mkdir ./train/cat
%mkdir ./train/dog
%mkdir ./test
%mkdir ./test/cat
%mkdir ./test/dog
%mkdir ./test/cat/data
%mkdir ./test/dog/data

In [ ]:
import os
from os import listdir
from os.path import isfile, join
import shutil

#split data in train folder to cat and dog folders
mypath = './train'

onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

for file in onlyfiles:
    if "cat" in file: 
        shutil.move(f"{mypath}/{file}", f"{mypath}/cat/{file}")
    if "dog" in file:
        shutil.move(f"{mypath}/{file}", f"{mypath}/dog/{file}")
        
#move images from ./train/cat to ./test/data/cat
mypath = './train/cat'
dem = 0

onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

for file in onlyfiles:
    dem += 1
    if dem <= 6250: shutil.move(f"{mypath}/{file}", f"./test/cat/data/{file}")
    else: break
    
#move images from ./train/dog to ./test/data/dog
mypath = './train/dog'
dem = 0

onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

for file in onlyfiles:
    dem += 1
    if dem <= 6250: shutil.move(f"{mypath}/{file}", f"./test/dog/data/{file}")
    else: break

**This is visualization of file tree after setup**

```
.
|__ train:
    |______ cats: [cat.0.jpg, cat.1.jpg ...]
    |______ dogs: [dog.0.jpg, dog.1.jpg ...]
|__ test:
    |______ cats: 
            |____ data: [cat.0.jpg, cat.1.jpg ...]
    |______ dogs: 
            |____ data: [dog.0.jpg, dog.1.jpg ...]
```

# **III. Import neccessary library**

In [ ]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, Activation, MaxPooling2D, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model

import os
import numpy as np
import matplotlib.pyplot as plt
import random
import math
import pandas as pd

# Variables for pre-processing and training.
batch_size = 128
epochs = 20
IMG_HEIGHT = 224
IMG_WIDTH = 224

# **IV. Initially set up data and model**

## **1. Create image generators for train image set and validation image set**

* We split images in train folder. 75% for training and 25% fot validation.
* Both train data generator and valid data generator have rescale 1./255 so Saturate problems can be reduced.
* Image Data Augmentation: Image data augmentation is a technique that can be used to artificially expand the size of a training dataset by creating modified versions of images in the dataset. Example: Rotate image, Zoom Image, Brightness, ...

In [ ]:
validation_generator = ImageDataGenerator(rescale=1./255, validation_split=0.25)
train_generator = ImageDataGenerator(rescale=1./255, rotation_range=15, width_shift_range=0.1, height_shift_range=0.1, brightness_range=(0.7, 1.3), shear_range=10.0, zoom_range=0.2, horizontal_flip=True, validation_split=0.25)
train_data_gen = train_generator.flow_from_directory(directory="train", target_size=(IMG_HEIGHT, IMG_WIDTH), batch_size=batch_size, shuffle=True, class_mode='binary', subset="training")
val_data_gen = validation_generator.flow_from_directory(directory="train", target_size=(IMG_HEIGHT, IMG_WIDTH), batch_size=batch_size, shuffle=True, class_mode='binary', subset="validation")

### **Plot image function**

In [ ]:
#Plot image function
def plotImages(images_arr):
    fig = plt.figure(figsize=(3, 3 * len(images_arr)))
    dem = 0
    for img in images_arr:
        dem += 1
        fig.add_subplot(len(images_arr), 1, dem)
        plt.imshow(img)
    plt.show()

### **Test rotated image form**

In [ ]:
augmented_images = [train_data_gen[0][0][0] for i in range(5)]

plotImages(augmented_images)

augmented_images = [val_data_gen[0][0][0] for i in range(5)]

plotImages(augmented_images)

## **2. Make a model**

We use convolutional neural network model because it's good for computer vision problems. Tryout multiple models:

**First, we try a model with three VGG blocks. The final accuracy of model is 64%**
* Activation Function or Transfer Function is the way to get output out of node. We use Relu because it can solve the saturate problems.
* In the last layer, we use activation = 'sigmoid' for classifing ouput.

In [ ]:
#Three Block VGG Model: 64%
model = Sequential([
    #Block 1
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    MaxPooling2D(2, 2),
    #Block 2
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    #Block 3
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2,2),
    #Flatten and Dense layer
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=SGD(lr=0.001, momentum=0.9), loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

**Second, we try a model with three VGG blocks and dropout but the accuracy of model drop to 61%**
* Dropout: regulization method to prevent overfitting

In [ ]:
#Three Block VGG Model + Dropout: 61%
model = Sequential([
    #Block 1
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    MaxPooling2D(2, 2),
    #Block 2
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    #Block 3
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2,2),
    #Dropout, Dense and Flatten layers
    Dropout(0.2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=SGD(lr=0.001, momentum=0.9), loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

**Finally, we modify already existing VGG16 model to apply to our problem. The accuracy jump to 90%**
* VGG16 is a convolutional neural network trained on a subset of the ImageNet dataset, a collection of over 14 million images belonging to 22,000 categories. Because its pre-trained model, we don't need to train again

In [ ]:
# VGG16 model 90%

# load VGG16 base model
# Include_top is set to False, in order to exclude the model's fully-connected layers.
conv_base = VGG16(include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))

# because vgg16 is pre-trained model, no need to train again 
for layer in conv_base.layers:
    layer.trainable = False
    
# add flatten and dense layer
output = conv_base.output
output = Flatten()(output)
output = Dense(128, activation='relu')(output)
output = Dense(1, activation='sigmoid')(output)

# define new model
model = Model(inputs=conv_base.inputs, outputs=output)

# compile model
model.compile(optimizer=SGD(lr=0.001, momentum=0.9), loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

# **V. Train model**

In [ ]:
#Use model.fit to train model
history = model.fit(x=train_data_gen, steps_per_epoch=16, epochs=epochs, validation_data=val_data_gen, validation_steps=16, verbose=1)

**Need visualization to prevent problems like overfitting**

In [ ]:
#visualize the accuracy and loss of the model.
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(10, 10))

plt.subplot(2, 1, 1)
plt.plot(range(epochs), accuracy, label='Training Accuracy')
plt.plot(range(epochs), val_accuracy, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Accuracy Performance')

plt.subplot(2, 1, 2)
plt.plot(range(epochs), loss, label='Training Loss')
plt.plot(range(epochs), val_loss, label='Validation Loss')
plt.legend(loc='lower left')
plt.title('Loss Performance')

plt.show()

# **VI. Test model**

## **1. Create test data generator**

In [ ]:
#create test image generator
test_generator = ImageDataGenerator(rescale=1./255)
test_dog_gen = test_generator.flow_from_directory(directory = "test/dog", target_size=(IMG_HEIGHT, IMG_WIDTH), batch_size=batch_size, shuffle=False, class_mode='binary')
test_cat_gen = test_generator.flow_from_directory(directory = "test/cat", target_size=(IMG_HEIGHT, IMG_WIDTH), batch_size=batch_size, shuffle=False, class_mode='binary')

## **2. Random pick 1 image from test folder and print the prediction**

In [ ]:
#random cat or dog
type_num = random.randint(1, 2)
#1 for cat
#2 for dog

#get random image
img_num = random.randint(1, 6250)
x_num = math.floor(img_num / batch_size)
y_num = img_num % batch_size

if type_num == 1: sample_test_images, _ = test_cat_gen[x_num]
else: sample_test_images, _ = test_dog_gen[x_num]
plotImages([sample_test_images[y_num]])

#print prediction
output = model.predict(np.array([sample_test_images[y_num]]))
x = output[0][0]
if (x > 0.5): print("dog")
else: print("cat")

## **3. Model performance**

In [ ]:
dog_predictions = model.predict(test_dog_gen)
cat_predictions = model.predict(test_cat_gen)
sum1 = sum(1 if dog > 0.5 else 0 for dog in dog_predictions)
sum2 = sum(1 if cat <= 0.5 else 0 for cat in cat_predictions)
print(f"Your model correctly identified {round((sum1 + sum2)/12500, 2) * 100}% of the images")